In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from pyquery import PyQuery as pq
from config import *
import pymongo
import re

ModuleNotFoundError: No module named 'selenium'

In [46]:
def search( keyword ):
    try:
        browser.get('https://www.jd.hk/')
        input = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR,"#key"))
        )
        submit = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,"#search-2014 > div.form > button")))
        input.send_keys(keyword)#搜索 keyword  = "口红"
        submit.click()
        total = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"#J_bottomPage > span.p-skip > em:nth-child(1) > b")))#总页数
        return total.text
    except TimeoutException:
        return search()

In [47]:
def next_page(page_number):
    try:
        input = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#page_jump_num"))
        )
        input.clear()
        input.send_keys(page_number)
        submit = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#page_jump_num")))
        submit.send_keys(Keys.ENTER)
        wait.until(EC.text_to_be_present_in_element((By.CSS_SELECTOR,"#J_bottomPage > span.p-num > a.curr"),str(page_number)))
        
        # get the current page info
        get_product()
    except TimeoutException:
        next_page(page_number)

In [48]:
def get_product():
    try:
        print("Ready to get product info:")
        # wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"#goods-list .gl-warp .gl-item")))
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"#plist > ul > li:nth-child(1)")))
        print("Page loaded successfully")
        html = browser.page_source
        doc = pq(html)
        items = doc(".gl-item").items()

        for item in items:          
            product = {
                # remove J_
                'SKU':item.find(".price").children().attr("psitemsku")[2:],
                # unaccessible
                'ImageUrl': item.find(".p-img").children().children().attr("src"),
                'Price':item.find(".price").text()[2:],
                'ProductName': re.sub("\n|全球购","",re.sub(item.find(".name-other").text(),"", item.find(".p-name").text())),
                'NameOther':item.find('.name-other').text(),
                'ProductUrl': item.find('.p-name').children().attr('href'),
                'Conduct':item.find(".p-conduct").text(),
                'CommentNum':re.sub("\n条评价","",item.find(".assess").text()),
                'ShopName':item.find(".p-shop-name").text(),
                'ShopNameUrl':item.find(".p-shop-name-text").attr("href"),
                'ShopChat':item.find(".p-shop-name-icon").attr("href"),
                'Picons':item.find('.p-icons').text()
            }
        
            save_to_mongo(product)
            # save_to_csv(product)
    except TimeoutException:
        return  get_product()

In [49]:
# def save_to_csv(product):

#     with open("product.csv",'w') as f:
#         s=product['image']+','+product['price']+','+product['name']+','+product['product_url']+','+product['conduct']+','+product['comment_num']+','+product['shop_name'] +'\n'
#         try:
#             f.write(s)
#             print('保存到csv成功！',product)
#         except:
#             pass
#数据保存到mongodb中
def save_to_mongo(result):
    
    #global
    global item_count
    item_count += 1
    try:
        if db[MONGO_TABLE].insert(result):
            print('NO:' + str(item_count) +' 插入成功', result)
    except Exception:
        print('NO:' + str(item_count) +' 插入失败', result)


In [50]:
def main():
    try:
        
        keyword = "口红"
        total = int(search(keyword))
        print('Total: ' + str(total) )
        
        for index in range(1,total+1):
            next_page(index)
#         for i in range(1,10):
#             next_page(i)
    except Exception:
        print("出错啦")
    finally:
        browser.close()
        client.close()


In [51]:
#View 
item_count = 0

client = pymongo.MongoClient(MONGO_URL)   # 创建一个连接对象
db = client[MONGO_DB]

MONGO_TABLE = 'lipstick_demo'

options=webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
browser = webdriver.Chrome(chrome_options=options)
wait = WebDriverWait(browser,10)

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
if __name__ == '__main__':
    main()